In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

from sklearn.model_selection import train_test_split, StratifiedKFold # crossvalidation wrt class ratio
from sklearn.preprocessing import MinMaxScaler, StandardScaler, scale
from sklearn.metrics import roc_auc_score

from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPool2D, BatchNormalization, Input, Conv2D
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam
from keras import backend as K
import keras
from keras.models import Model
from keras import regularizers
import tensorflow as tf
from keras.losses import binary_crossentropy
import gc # trash return
import scipy.special
from tqdm import *  # processing bar
from scipy.stats import norm, rankdata

from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

BATCH_SIZE = 1024
NUM_FEATURES = 600

['train.csv', 'sample_submission.csv', 'test.csv']


Using TensorFlow backend.


## 1-Load dataset and preprocessing

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
# inspect data
print('train.shape', train.shape, '\ntest.shape', test.shape)

train.shape (200000, 202) 
test.shape (200000, 201)


In [4]:
print ( train.info())
print ( test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 201 entries, ID_code to var_199
dtypes: float64(200), object(1)
memory usage: 306.7+ MB
None


In [5]:
train.isnull().any().describe()


count       202
unique        1
top       False
freq        202
dtype: object

In [6]:
test.isnull().any().describe()

count       201
unique        1
top       False
freq        201
dtype: object

In [7]:
train[10:20]

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
10,train_10,0,5.0702,-0.5447,9.5900,4.2987,12.3910,-18.8687,6.0382,14.3797,-0.4711,7.3198,4.6603,-14.0548,13.9059,9.0796,11.8218,14.3125,11.0386,4.7462,17.4442,10.7502,11.6244,8.2466,-0.5277,2.8072,7.4451,13.7830,-0.2494,-0.1141,5.9583,2.8036,-0.2332,12.5400,2.1982,11.5565,11.0755,5.4034,8.5073,6.9892,...,7.0656,5.3470,3.7800,8.0339,-9.9638,20.9325,3.7723,-3.8352,7.4966,5.7487,0.4344,6.5370,7.2105,-3.5162,27.1062,13.2255,1.1165,10.2981,-4.0424,3.9860,-11.6023,11.1783,14.8384,10.9622,22.4111,-3.3333,16.2396,-7.0216,17.3210,0.6963,7.2780,8.0819,0.2139,-0.0761,14.9585,-1.2160,6.6576,9.2553,14.2914,-7.6652
11,train_11,0,12.7188,-7.9750,10.3757,9.0101,12.8570,-12.0852,5.6464,11.8370,1.2953,6.8093,-6.1501,-5.4925,13.6713,9.5331,4.8230,14.7383,6.8414,-9.0195,17.5407,17.1926,6.0760,2.3546,7.6435,3.9800,6.9099,13.3506,-8.0681,-3.9470,3.9651,7.7577,-6.7792,11.9451,-3.2295,10.7779,10.8248,-5.8872,-0.2503,6.7759,...,5.9874,5.4122,3.0756,12.5066,1.5809,18.2420,3.6452,-3.6111,3.4261,5.7226,2.7449,-0.3629,23.9603,-4.3861,22.4637,10.8169,-7.0860,13.8536,-5.3953,6.3811,-0.2817,11.1792,3.9031,14.1906,14.7535,-10.4444,6.8885,-0.1458,21.8375,-0.1857,-0.8901,2.6559,-0.0503,5.5946,13.6152,2.4058,-1.7183,9.6745,16.7498,-3.9728
12,train_12,0,8.7671,-4.6154,9.7242,7.4242,9.0254,1.4247,6.2815,12.3143,5.6964,6.0197,5.2524,-4.5162,14.1985,9.6978,8.6948,13.9111,9.3556,-13.7217,2.1749,11.4645,9.4219,34.8112,6.4927,4.0522,12.5483,13.5935,-9.9662,-2.0558,5.6994,11.3379,4.0196,8.3829,3.6990,23.8890,12.0540,0.9791,0.7492,5.3935,...,30.5234,5.7813,6.4252,20.5597,-3.8090,20.3979,3.3998,-12.2034,8.4840,5.4729,3.2530,4.7714,18.5149,-1.3783,27.0374,10.3448,-7.8723,14.0301,-14.0628,-1.2209,8.6217,10.2952,-16.5506,6.9453,-0.5242,-1.3825,10.0550,-13.6977,20.0393,1.2974,0.3782,7.4382,0.0854,1.3444,17.2439,-0.0798,5.7389,8.4897,17.0938,4.6106
13,train_13,1,16.3699,1.5934,16.7395,7.3330,12.1450,5.9004,4.8222,20.9729,1.1064,8.6978,2.3287,-11.3409,13.7999,2.7925,6.3182,14.7313,7.2594,-2.4759,14.3984,9.1793,16.8467,19.4258,1.6565,2.5107,7.1272,13.6444,-12.8761,-1.0136,5.1773,7.3507,-16.0042,6.4460,-3.3238,14.3372,10.9481,6.7386,-0.5110,4.4335,...,32.5287,5.8451,4.9591,11.7805,-7.0936,11.8863,3.2597,-4.3478,7.5711,5.6656,3.5750,-8.6583,20.3131,-2.2723,15.7383,8.1730,-12.4963,15.1929,3.9751,8.5618,2.4566,9.3694,0.6942,11.9694,11.8982,0.6129,10.8123,-1.0803,11.9586,-0.5899,7.4002,7.4031,4.3989,4.0978,17.3638,-1.3022,9.6846,9.0419,15.6064,-10.8529
14,train_14,0,13.8080,5.0514,17.2611,8.5120,12.8517,-9.1622,5.7327,21.0517,-4.5117,6.8116,8.2028,-7.8221,13.9241,10.3896,6.8838,14.1297,14.5268,-4.8877,26.1382,4.8558,23.4624,20.8922,8.0659,4.0123,7.6241,13.3559,-13.1966,-3.8157,6.1292,5.2018,-1.6535,11.3707,2.7550,19.5342,10.6458,10.0454,0.9564,3.8333,...,21.0429,5.7070,7.2320,13.8567,8.0575,9.5069,2.8172,-10.9940,4.0863,5.1196,-2.4897,3.1915,15.8254,3.8015,26.2032,8.5550,5.5689,10.5646,-1.1600,3.2684,-4.2496,10.5889,-15.4406,1.9688,30.8736,-6.4666,4.4271,16.0274,18.4116,1.0279,1.0740,8.3220,3.2619,1.6738,17.4797,-0.0257,-3.5323,9.3439,24.4479,-5.1110
15,train_15,0,3.9416,2.6562,13.3633,6.8895,12.2806,-16.1620,5.6979,14.4573,-4.3144,7.1290,-7.0984,1.7324,14.1446,5.2403,5.0599,14.6456,7.2626,-15.3607,24.7424,17.6439,11.5724,11.4583,0.5490,2.3395,8.3642,14.2083,-7.7781,-2.0180,6.4194,3.6106,-12.7855,11.1948,1.3230,13.9853,10.7737,7.0264,1.6508,6.0989,...,17.7009,5.4925,4.6479,10.2630,1.9567,11.3951,2.

In [8]:
y = train['target']
id_code_train = train['ID_code']
id_code_test = test['ID_code']
features = [c for c in train.columns if c not in ['ID_code','target']]

### 1.1 Normalization

In [9]:
class GaussRankScaler():
    def __init__(self):
        self.epsilon = 1e-9
        self.lower = -1+self.epsilon
        self.upper = 1 +self.epsilon
        self.range = self.upper-self.lower
        
    def fit_transform(self,X):
        i = np.argsort(X, axis=0)
        j = np.argsort(i, axis=0)
        
        assert(j.min()==0).all()
        assert(j.max()==len(j)-1).all()
        
        j_range = len( j ) - 1
        self.divider = j_range / self.range

        transformed = j / self.divider
        transformed = transformed - self.upper
        transformed = scipy.special.erfinv( transformed )
        ############
        # transformed = transformed - np.mean(transformed)

        return transformed

In [10]:
SPLIT = len(train)
train = train.append(test)
del test; gc.collect()  # trash 

scaler = GaussRankScaler()
sc = StandardScaler()

for feat in tqdm(features):
    train[feat] = sc.fit_transform(train[feat].values.reshape(-1,1))
    train[feat+'_r'] = rankdata(train[feat]).astype('float32')
    train[feat+'_n'] = norm.cdf(train[feat]).astype('float32')
    
feats = [c for c in train.columns if c not in (['ID_code', 'target']+features)]

for feat in tqdm(feats):
    train[feat] = sc.fit_transform(train[feat].values.reshape(-1,1))
    
train = train.drop(['target','ID_code'], axis=1)
test = train[SPLIT:].values
train = train[:SPLIT].values

print('Done')
print(train.shape)
print(test.shape)
#train[0:5]


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
100%|██████████| 400/400 [00:02<00:00, 153.45it/s]


Done
(200000, 600)
(200000, 600)


In [11]:
train = np.reshape(train,(-1, NUM_FEATURES, 1))
test = np.reshape(test,(-1, NUM_FEATURES, 1))

In [12]:
x_train, x_valid, y_train, y_valid = train_test_split(train, y, stratify=y, test_size=0.2, random_state=8)

## 2-Network

In [13]:
function = keras.layers.advanced_activations.LeakyReLU(alpha=.001)
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    x = Dense(16, activation=function)(input_tensor)
    x = Flatten()(x)
    out_put = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, out_put)
    
    return model


def auc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

gamma = 2.0
alpha=.25
epsilon = K.epsilon()

def focal_loss(y_true, y_pred):
    pt_1 = y_pred * y_true
    pt_1 = K.clip(pt_1, epsilon, 1-epsilon)
    CE_1 = -K.log(pt_1)
    FL_1 = alpha* K.pow(1-pt_1, gamma) * CE_1
    
    pt_0 = (1-y_pred) * (1-y_true)
    pt_0 = K.clip(pt_0, epsilon, 1-epsilon)
    CE_0 = -K.log(pt_0)
    FL_0 = (1-alpha)* K.pow(1-pt_0, gamma) * CE_0
    
    loss = K.sum(FL_1, axis=1) + K.sum(FL_0, axis=1)
    return loss

def mixup_data(x, y, alpha=1.0):
    # y = np.array(y)
    # print(y)
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    sample_size = x.shape[0]
    index_array = np.arange(sample_size)
    np.random.shuffle(index_array)
    
    mixed_x = lam * x + (1 - lam) * x[index_array]
    mixed_y = (lam * y) + ((1 - lam) * y[index_array])
    # print((1 - lam) * y[index_array])
    # print((lam * y).shape,((1 - lam) * y[index_array]).shape)
    return mixed_x, mixed_y

def make_batches(size, batch_size):
    nb_batch = int(np.ceil(size/float(batch_size)))
    return [(i*batch_size, min(size, (i+1)*batch_size)) for i in range(0, nb_batch)]


def batch_generator(X,y,batch_size=128,shuffle=True,mixup=False):
    y = np.array(y)
    # print(X.shape[0], y.shape[0])
    sample_size = X.shape[0]
    index_array = np.arange(sample_size)
    
    while True:
        if shuffle:
            np.random.shuffle(index_array)
        batches = make_batches(sample_size, batch_size)
        for batch_index, (batch_start, batch_end) in enumerate(batches):
            batch_ids = index_array[batch_start:batch_end]
            X_batch = X[batch_ids]
            y_batch = y[batch_ids]
            
            if mixup:
                # print('before', X_batch.shape, y_batch.shape)
                X_batch,y_batch = mixup_data(X_batch,y_batch,alpha=1.0)
            # print('*****************')    
            yield X_batch,y_batch

## 3- Feed forward NN

In [14]:
model = create_model((NUM_FEATURES,1),1)
model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()

checkpoint = ModelCheckpoint('feed_forward_model.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, 
                                   verbose=1, mode='min', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=9)
callbacks_list = [checkpoint, reduceLROnPlat, early]
tr_gen = batch_generator(x_train,y_train,batch_size=BATCH_SIZE,shuffle=True,mixup=True)

history = model.fit_generator(# x_train,y_train,
                                tr_gen,
                                steps_per_epoch=np.ceil(float(len(x_train)) / float(BATCH_SIZE)),
                                epochs=20,
                                verbose=1,
                                callbacks=callbacks_list,
                                validation_data=(x_valid, y_valid))

/opt/conda/lib/python3.6/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))
/opt/conda/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 600, 1)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 600, 16)           32        
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9601      
Total params: 9,633
Trainable params: 9,633
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
157/157 [==============================] - 23s 143ms/step - loss: 0.3061 - val_loss: 0.2529

Epoch 00001: val_loss improved from inf to 0.25294, saving model to feed_forward_model.h5
Epoch 2/20
157/157 [==============================] - 22s 138ms/step - loss: 0

In [15]:
model.load_weights('feed_forward_model.h5')
prediction = model.predict(x_valid, batch_size=512, verbose=1)
roc_auc_score(y_valid, prediction)

40000/40000 [==============================] - 1s 30us/step


0.8842479618306468

## 4- make submission

In [16]:
predict = model.predict(test, batch_size=512, verbose=1)

200000/200000 [==============================] - 6s 30us/step


In [17]:
predict

array([[0.13750668],
       [0.19902281],
       [0.11948104],
       ...,
       [0.00236215],
       [0.06136876],
       [0.07921867]], dtype=float32)

In [18]:
predict1 = np.round(predict,0).astype(int)

In [19]:
predict1

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [20]:
# make submission
submit=pd.read_csv('../input/sample_submission.csv')
submit['ID_code'] = id_code_test
submit['target'] = predict1
submit.to_csv('submission.csv', index=False)
submit.head()

,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0
